# 四、 变形
1. **透视表**  
pivot  
pivot_table  
crosstab  
  
  
2. **其他变形方法**  
melt  
压缩与展开  
  
  
3. **哑变量与因子化**  
Dummy Variable（哑变量）  
factorize方法  
   

In [1]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df = pd.read_csv('./data/table.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   School   35 non-null     object 
 1   Class    35 non-null     object 
 2   ID       35 non-null     int64  
 3   Gender   35 non-null     object 
 4   Address  35 non-null     object 
 5   Height   35 non-null     int64  
 6   Weight   35 non-null     int64  
 7   Math     35 non-null     float64
 8   Physics  35 non-null     object 
dtypes: float64(1), int64(3), object(5)
memory usage: 2.6+ KB


## 1. 透视表

### pivot
一般状态下，数据在DataFrame会以压缩（stacked）状态存放，例如上面的Gender，两个类别被叠在一列中，pivot函数可将某一列作为新的cols：  
类似excel中的数据透视表

In [2]:
df.pivot(index='ID', columns='Gender', values='Height').head()

Gender,F,M
ID,,
1101,NaN,173.0
1102,192.0,NaN
1103,NaN,186.0
1104,167.0,NaN
1105,159.0,NaN


In [ ]:
#### 然而pivot函数具有很强的局限性，除了功能上较少之外，还不允许values中出现重复的行列索引对（pair），例如下面的语句就会报错：
#df.pivot(index='School',columns='Gender',values='Height').head()

### pivot_table

In [7]:
pd.pivot_table(df, index='ID', columns='Gender', values='Height').head()
pd.pivot_table(df, index='School', columns='Gender', values='Height').head()

Gender,F,M
ID,,
1101,NaN,173.0
1102,192.0,NaN
1103,NaN,186.0
1104,167.0,NaN
1105,159.0,NaN


Gender,F,M
School,,
S_1,173.125000,178.714286
S_2,173.727273,172.000000


In [8]:
# 由于功能更多，速度上自然是比不上原来的pivot函数：
%timeit df.pivot(index='ID',columns='Gender',values='Height')
%timeit pd.pivot_table(df,index='ID',columns='Gender',values='Height')

3.57 ms ± 965 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.5 ms ± 3.23 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
#Pandas中提供了各种选项，下面介绍常用参数：
# ① aggfunc：对组内进行聚合统计，可传入各类函数，默认为'mean'
pd.pivot_table(df,index='School',columns='Gender',values='Height',aggfunc=['sum', 'mean']).head()

# ② margins：汇总边际状态,即是否显示subtotal
pd.pivot_table(df,index='School',columns='Gender',values='Height',aggfunc=['sum','mean'],margins=True, margins_name='Subtoal').head()

# ③ 行、列、值都可以为多级
pd.pivot_table(df,index=['School','Class'],columns=['Gender','Address'],values=['Height','Weight'])
pd.pivot_table(df,index=['School','Class'],columns=['Gender'],values=['Height','Weight'], aggfunc=['sum','mean'])

sum              mean            
Gender     F     M           F           M
School                                    
S_1     1385  1251  173.125000  178.714286
S_2     1911  1548  173.727273  172.000000

sum                      mean                        
Gender      F     M Subtoal           F           M     Subtoal
School                                                         
S_1      1385  1251    2636  173.125000  178.714286  175.733333
S_2      1911  1548    3459  173.727273  172.000000  172.950000
Subtoal  3296  2799    6095  173.473684  174.937500  174.142857

Height                                                        \
Gender              F                                                     M   
Address      street_1 street_2 street_4 street_5 street_6 street_7 street_1   
School Class                                                                  
S_1    C_1        NaN    179.5    159.0      NaN      NaN      NaN    173.0   
       C_2        NaN      NaN    176.0    162.0    167.0      NaN      NaN   
       C_3      175.0      NaN      NaN    187.0      NaN      NaN      NaN   
S_2    C_1        NaN      NaN      NaN    159.0    161.0      NaN      NaN   
       C_2        NaN      NaN      NaN      NaN      NaN    188.5    175.0   
       C_3        NaN      NaN    157.0      NaN    164.0    190.0      NaN   
       C_4        NaN    176.0      NaN      NaN    175.5      NaN      NaN   

                                         ...   Weight                    \
Gender                                   ...        F                     
Address      street_2 street_4 street_5  ... street_4 street_5 street_6   
School Class                             ...                              
S_1    C_1      186.0      NaN      NaN  ...     64.0      NaN      NaN   
       C_2        NaN      NaN    188.0  ...     94.0     63.0     63.0   
       C_3      195.0    161.0      NaN  ...      NaN     69.0      NaN   
S_2    C_1        NaN    163.5      NaN  ...      NaN     97.0     61.0   
       C_2        NaN    155.0    193.0  ...      NaN      NaN      NaN   
       C_3        NaN    187.0    171.0  ...     78.0      NaN     81.0   
       C_4        NaN      NaN      NaN  ...      NaN      NaN     57.0   

                                                                             
Gender                       M                                               
Address      street_7 street_1 street_2 street_4 street_5 street_6 street_7  
School Class                                                                 
S_1    C_1        NaN     63.0     82.0      NaN      NaN      NaN      NaN  
       C_2        NaN      NaN      NaN      NaN     68.0     53.0      NaN  
       C_3        NaN      NaN     70.0     68.0      NaN      NaN     82.0  
S_2    C_1        NaN      NaN      NaN     71.0      NaN      NaN     84.0  
       C_2       76.5     74.0      NaN     91.0    100.0      NaN      NaN  
       C_3       99.0      NaN      NaN     73.0     88.0      NaN      NaN  
       C_4        NaN      NaN      NaN      NaN      NaN      NaN     82.0  

[7 rows x 24 columns]

sum                         mean                         \
             Height      Weight           Height                 Weight   
Gender            F    M      F    M           F           M          F   
School Class                                                              
S_1    C_1      518  359    218  145  172.666667  179.500000  72.666667   
       C_2      505  348    220  121  168.333333  174.000000  73.333333   
       C_3      362  544    126  220  181.000000  181.333333  63.000000   
S_2    C_1      320  501    158  226  160.000000  167.000000  79.000000   
       C_2      377  523    153  265  188.500000  174.333333  76.500000   
       C_3      511  358    258  161  170.333333  179.000000  86.000000   
       C_4      703  166    260   82  175.750000  166.000000  65.000000   

                         
                         
Gender                M  
School Class             
S_1    C_1    72.500000  
       C_2    60.500000  
       C_3    73.333333  
S_2    C_1    75.333333  
       C_2    88.333333  
       C_3    80.500000  
       C_4    82.000000

### crosstab 

In [10]:
# 交叉表是一种特殊的透视表，典型的用途如分组统计，如现在想要统计关于街道和性别分组的频数：
pd.crosstab(index=df['Address'], columns=df['Gender'])

Gender,F,M
Address,,
street_1,1,2
street_2,4,2
street_4,3,5
street_5,3,3
street_6,5,1
street_7,3,3


In [26]:
# 交叉表的功能也很强大（但目前还不支持多级分组），下面说明一些重要参数：
## ① values和aggfunc：分组对某些数据进行聚合操作，这两个参数必须成对出现
pd.crosstab(index=df['Address'],columns=df['Gender'],values=np.random.randint(1,20,df.shape[0]),aggfunc='min')
pd.crosstab(index=df['Address'],columns=df['Gender'],values=df['Math'],aggfunc='mean')
# np.random.randint(1,20,df.shape[0]) 随机数无意义

## ② 除了边际参数margins外，还引入了normalize参数，可选'all','index','columns'参数值
pd.crosstab(index=df['Address'],columns=df['Gender'],normalize='all',margins=True)
pd.crosstab(index=df['Address'],columns=df['Gender'],normalize='index',margins=True)
pd.crosstab(index=df['Address'],columns=df['Gender'],normalize='columns',margins=True)
pd.crosstab(index=df['Address'],columns=df['Gender'],normalize=True) # 和第一个等价
# 如果希望看到的是占比情况，这时只需设置crosstab()方法中的normalize参数即可。



Gender,F,M
Address,,
street_1,7,13
street_2,6,13
street_4,1,5
street_5,1,6
street_6,3,18
street_7,6,4


Gender,F,M
Address,,
street_1,87.700000,40.600000
street_2,56.475000,86.200000
street_4,73.533333,48.180000
street_5,55.900000,56.266667
street_6,64.360000,58.800000
street_7,73.266667,60.566667


Gender,F,M,All
Address,,,
street_1,0.028571,0.057143,0.085714
street_2,0.114286,0.057143,0.171429
street_4,0.085714,0.142857,0.228571
street_5,0.085714,0.085714,0.171429
street_6,0.142857,0.028571,0.171429
street_7,0.085714,0.085714,0.171429
All,0.542857,0.457143,1.000000


Gender,F,M
Address,,
street_1,0.333333,0.666667
street_2,0.666667,0.333333
street_4,0.375000,0.625000
street_5,0.500000,0.500000
street_6,0.833333,0.166667
street_7,0.500000,0.500000
All,0.542857,0.457143


Gender,F,M,All
Address,,,
street_1,0.052632,0.1250,0.085714
street_2,0.210526,0.1250,0.171429
street_4,0.157895,0.3125,0.228571
street_5,0.157895,0.1875,0.171429
street_6,0.263158,0.0625,0.171429
street_7,0.157895,0.1875,0.171429


Gender,F,M
Address,,
street_1,0.028571,0.057143
street_2,0.114286,0.057143
street_4,0.085714,0.142857
street_5,0.085714,0.085714
street_6,0.142857,0.028571
street_7,0.085714,0.085714


## 2. 其他变形方法

### melt

In [30]:
# melt函数可以认为是pivot函数的逆操作，将unstacked状态的数据，压缩成stacked，使“宽”的DataFrame变“窄”
# 逆透视
df_m = df[['ID','Gender','Math']]
df_m.head()

pivoted = df.pivot(index='ID',columns='Gender',values='Math')
result = pivoted.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math').dropna().set_index('ID').sort_index()
# 感觉略复杂啊，要多次重排索引
# 检验是否与展开前的df相同，可以分别将这些链式方法的中间步骤展开，看看是什么结果
result.equals(df_m.set_index('ID'))

,ID,Gender,Math
0,1101,M,34.0
1,1102,F,32.5
2,1103,M,87.2
3,1104,F,80.4
4,1105,F,84.8


True

In [31]:
pivoted.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math')
pivoted.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math').dropna()
pivoted.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math').dropna().set_index('ID')
pivoted.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math').dropna().set_index('ID').sort_index()

,ID,Gender,Math
0,1101,F,NaN
1,1102,F,32.5
2,1103,F,NaN
3,1104,F,80.4
4,1105,F,84.8
...,...,...,...
65,2401,M,NaN
66,2402,M,48.7
67,2403,M,NaN
68,2404,M,NaN


,ID,Gender,Math
1,1102,F,32.5
3,1104,F,80.4
4,1105,F,84.8
6,1202,F,63.5
8,1204,F,33.8
9,1205,F,68.4
11,1302,F,87.7
14,1305,F,61.7
16,2102,F,50.6
18,2104,F,72.2


,Gender,Math
ID,,
1102,F,32.5
1104,F,80.4
1105,F,84.8
1202,F,63.5
1204,F,33.8
1205,F,68.4
1302,F,87.7
1305,F,61.7
2102,F,50.6


,Gender,Math
ID,,
1101,M,34.0
1102,F,32.5
1103,M,87.2
1104,F,80.4
1105,F,84.8
1201,M,97.0
1202,F,63.5
1203,M,58.8
1204,F,33.8


### 压缩与展开 

In [34]:
# 1）stack：这是最基础的变形函数，总共只有两个参数：level和dropna
df_s = pd.pivot_table(df,index=['Class','ID'],columns='Gender',values=['Height','Weight'])
df_s.groupby('Class').head(2)

df_stacked = df_s.stack()
df_stacked.groupby('Class').head(2)

## stack函数可以看做将横向的索引放到纵向，因此功能类似与melt，参数level可指定变化的列索引是哪一层（或哪几层，需要列表）
df_stacked = df_s.stack(0)
df_stacked.groupby('Class').head(2)

Height        Weight      
Gender          F      M      F     M
Class ID                             
C_1   1101    NaN  173.0    NaN  63.0
      1102  192.0    NaN   73.0   NaN
C_2   1201    NaN  188.0    NaN  68.0
      1202  176.0    NaN   94.0   NaN
C_3   1301    NaN  161.0    NaN  68.0
      1302  175.0    NaN   57.0   NaN
C_4   2401  192.0    NaN   62.0   NaN
      2402    NaN  166.0    NaN  82.0

Height  Weight
Class ID   Gender                
C_1   1101 M        173.0    63.0
      1102 F        192.0    73.0
C_2   1201 M        188.0    68.0
      1202 F        176.0    94.0
C_3   1301 M        161.0    68.0
      1302 F        175.0    57.0
C_4   2401 F        192.0    62.0
      2402 M        166.0    82.0

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
C_2   1201 Height    NaN  188.0
           Weight    NaN   68.0
C_3   1301 Height    NaN  161.0
           Weight    NaN   68.0
C_4   2401 Height  192.0    NaN
           Weight   62.0    NaN

In [35]:
# 2) unstack：stack的逆函数，功能上类似于pivot_table
df_stacked.head()

result = df_stacked.unstack().swaplevel(1,0,axis=1).sort_index(axis=1)
result.equals(df_s)
#同样在unstack中可以指定level参数

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
      1102 Height  192.0    NaN
           Weight   73.0    NaN
      1103 Height    NaN  186.0

True

## 3. 哑变量与因子化

### Dummy Variable（哑变量）
其实就是给分类变量编码，主要思想有：  
1.单纯将文本变成数字  
2.每个类别生成一列

In [41]:
df_d = df[['Class','Gender','Weight']]
pd.get_dummies(df_d[['Class','Gender']]).join(df_d['Weight']).head() # 还需要和原始数据拼接起来，原始数据里要去掉dummies列
#可选prefix参数添加前缀，prefix_sep添加分隔符
pd.get_dummies(df_d[['Class','Gender']], prefix='test', prefix_sep='-').head()

,Class_C_1,Class_C_2,Class_C_3,Class_C_4,Gender_F,Gender_M,Weight
0,1,0,0,0,0,1,63
1,1,0,0,0,1,0,73
2,1,0,0,0,0,1,82
3,1,0,0,0,1,0,81
4,1,0,0,0,1,0,64


,test-C_1,test-C_2,test-C_3,test-C_4,test-F,test-M
0,1,0,0,0,0,1
1,1,0,0,0,1,0
2,1,0,0,0,0,1
3,1,0,0,0,1,0
4,1,0,0,0,1,0


### factorize方法 

In [42]:
# 方法主要用于自然数编码，并且缺失值会被记做-1，其中sort参数表示是否排序后赋值
codes, uniques = pd.factorize(['b', None, 'a', 'c', 'b'], sort=True)
display(codes)
display(uniques)

array([ 1, -1,  0,  2,  1], dtype=int64)

array(['a', 'b', 'c'], dtype=object)

## 4. 问题与联系
### 1. 问题

#### 【问题一】 
**上面提到了许多变形函数，如melt/crosstab/pivot/pivot_table/stack/unstack函数，请总结它们各自的使用特点。**  
melt是pivot的逆操作  
stack/unstack 针对索引进行操作，  
pivot/pivot_table 针对值进行操作  
crosstab不支持多级分组；其他的支持对多列操作  
    
    
#### 【问题二】 
**变形函数和多级索引是什么关系？哪些变形函数会使得索引维数变化？具体如何变化？**  
索引的本质是什么：  
索引应该是从数据库引申而来的概念，  
MySQL官方对索引的定义为：索引（Index）是帮助MySQL高效获取数据的数据结构。简单说：索引是一种数据结构。  
索引是一种数据结构。索引中是包含一个表中列的值和它的物理地址的值，并且这些值存储在一个数据结构中。  
作用是增加查询效率  
  
变形函数的作用主要是汇总统计数据，只不过统计的维度恰好以多级索引的形式呈现  
支持多列操作的都有可能导致索引维数变化  
  
  
#### 【问题三】 
**请举出一个除了上文提过的关于哑变量方法的例子。**  
OneHotEncoder  
  
#### 【问题四】 
**使用完stack后立即使用unstack一定能保证变化结果与原始表完全一致吗？**  
不一定，索引顺序可能会发生变化。  
  
#### 【问题五】 
**透视表中涉及了三个函数，请分别使用它们完成相同的目标（任务自定）并比较哪个速度最快。**
  
  


In [9]:
%timeit df.pivot(index='ID',columns='Gender',values='Height')
%timeit pd.pivot_table(df,index='ID',columns='Gender',values='Height')
%timeit pd.crosstab(index=df['ID'],columns=df['Gender'])

2.4 ms ± 175 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.2 ms ± 366 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
11.7 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### 【问题六】 
**既然melt起到了stack的功能，为什么再设计stack函数？**  
stack有个使用前提，需要对普通列设置为索引，在stack之后又要将这些索引转换为普通列，未免过于麻烦。  
应该stack和unstack是成对出现的；melt在简单场景下使用更方便 

#### 【参考资料】：
1. [pandas中DataFrame的stack()、unstack()和pivot()方法的对比](https://blog.csdn.net/S_o_l_o_n/article/details/80917211)
2. [数据库（一）-------索引的本质、类型以及如何创建索引详解（基础）](https://blog.csdn.net/qq_36098284/article/details/79841094) 

### 2. 练习

#### 【练习一】 
继续使用上一章的药物数据集：


In [81]:
drug = pd.read_csv('./data/Drugs.csv')
drug.head()
drug.info()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24062 entries, 0 to 24061
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   YYYY           24062 non-null  int64 
 1   State          24062 non-null  object
 2   COUNTY         24062 non-null  object
 3   SubstanceName  24062 non-null  object
 4   DrugReports    24062 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 940.0+ KB


 (a) 现在请你将数据表转化成如下形态，每行需要显示每种药物在每个地区的10年至17年的变化情况，且前三列需要排序：
![avatar](picture/drug_pic.png)
 (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [58]:
result = pd.pivot_table(drug, index=['State','COUNTY','SubstanceName'],columns='YYYY', values='DrugReports',aggfunc='sum',fill_value='-')\
        .reset_index().sort_values(by=['State','COUNTY','SubstanceName'])
# reset_index() 会将多级索引变回数字索引

YYYY,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3
...,...,...,...,...,...,...,...,...,...,...,...
6209,WV,WOOD,Oxycodone,6,4,24,7,7,11,7,1
6210,WV,WOOD,Tramadol,-,-,-,-,1,-,4,3
6211,WV,WYOMING,Buprenorphine,-,1,1,1,-,-,-,1
6212,WV,WYOMING,Hydrocodone,1,5,-,-,1,-,1,-


In [85]:
result = pd.pivot_table(drug,index=['State','COUNTY','SubstanceName']
                 ,columns='YYYY'
                 ,values='DrugReports',fill_value='-').reset_index().rename_axis(columns={'YYYY':''})
result_melted = result.melt(id_vars=result.columns[:3],value_vars=result.columns[-8:]
                ,var_name='YYYY',value_name='DrugReports').query('DrugReports != "-"')
result2 = result_melted.sort_values(by=['State','COUNTY','YYYY','SubstanceName']).reset_index().drop(columns='index')
result2 # 索引变了
df_tidy = drug.reset_index().sort_values(by=result2.columns[:4].tolist()).reset_index().drop(columns='index')
df_tidy
df_tidy.equals(result2)

,State,COUNTY,SubstanceName,YYYY,DrugReports
0,KY,ADAIR,Hydrocodone,2010,6
1,KY,ADAIR,Methadone,2010,1
2,KY,ADAIR,Buprenorphine,2011,3
3,KY,ADAIR,Hydrocodone,2011,9
4,KY,ADAIR,Morphine,2011,2
...,...,...,...,...,...
24057,WV,WYOMING,Oxycodone,2013,12
24058,WV,WYOMING,Hydrocodone,2014,1
24059,WV,WYOMING,Oxycodone,2014,5
24060,WV,WYOMING,Hydrocodone,2016,1


,level_0,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2731,2011,KY,ADAIR,Buprenorphine,3
1,5319,2012,KY,ADAIR,Buprenorphine,5
2,8782,2013,KY,ADAIR,Buprenorphine,4
3,12163,2014,KY,ADAIR,Buprenorphine,27
4,13645,2015,KY,ADAIR,Buprenorphine,5
...,...,...,...,...,...,...
24057,2453,2010,WV,WYOMING,Oxycodone,5
24058,4976,2011,WV,WYOMING,Oxycodone,4
24059,8428,2012,WV,WYOMING,Oxycodone,14
24060,10926,2013,WV,WYOMING,Oxycodone,12


False

#### 【练习二】 
现有一份关于某地区地震情况的数据集，请解决如下问题：

In [94]:
earthquake = pd.read_csv('data/Earthquake.csv')
earthquake.head()
earthquake.info()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10062 entries, 0 to 10061
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      10062 non-null  object 
 1   时间      10062 non-null  object 
 2   维度      10062 non-null  float64
 3   经度      10062 non-null  float64
 4   方向      10062 non-null  object 
 5   距离      10062 non-null  float64
 6   深度      10062 non-null  float64
 7   烈度      10062 non-null  float64
dtypes: float64(5), object(3)
memory usage: 629.0+ KB


(a) 现在请你将数据表转化成如下形态，将方向列展开，并将距离、深度和烈度三个属性压缩：
![avatar](picture/earthquake_pic.png)
(b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [95]:
# 先按烈度 深度 距离piviot
step1 = pd.pivot_table(earthquake,index=['日期','时间','维度','经度'],columns='方向',values=['烈度','深度','距离'],fill_value='-')
# 再将深度 烈度 距离 收回来
step1.stack(level=0).reset_index()


方向,日期,时间,维度,经度,level_4,east,north,north_east,north_west,south,south_east,south_west,west
0,1912.08.09,12:29:00 AM,40.60,27.20,深度,-,-,-,-,-,16,-,-
1,1912.08.09,12:29:00 AM,40.60,27.20,烈度,-,-,-,-,-,6.7,-,-
2,1912.08.09,12:29:00 AM,40.60,27.20,距离,-,-,-,-,-,4.3,-,-
3,1912.08.10,12:23:00 AM,40.60,27.10,深度,-,-,-,-,-,-,15,-
4,1912.08.10,12:23:00 AM,40.60,27.10,烈度,-,-,-,-,-,-,6,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30181,2017.07.30,12:21:33 AM,37.39,31.37,烈度,-,-,-,-,-,-,3.5,-
30182,2017.07.30,12:21:33 AM,37.39,31.37,距离,-,-,-,-,-,-,6,-
30183,2017.07.31,12:11:31 AM,37.42,31.37,深度,-,-,-,5.1,-,-,-,-
30184,2017.07.31,12:11:31 AM,37.42,31.37,烈度,-,-,-,3.5,-,-,-,-


In [96]:
antistep1 = step1.stack(level=0).unstack() # 将深度 烈度 距离放回去
antistep2 = antistep1.stack(level=0).reset_index() # 将方向归位，并reset_index
# antistep2.drop_duplicates(['日期','时间','维度','经度','方向']) # 如何排除深度烈度距离都是 - 的
antistep2=antistep2[~antistep2['深度'].isin(['-'])] # 排除掉 - 
antistep2.reset_index()


,index,日期,时间,维度,经度,方向,深度,烈度,距离
0,5,1912.08.09,12:29:00 AM,40.60,27.20,south_east,16,6.7,4.3
1,14,1912.08.10,12:23:00 AM,40.60,27.10,south_west,15,6,2
2,22,1912.08.10,12:30:00 AM,40.60,27.10,south_west,15,5.2,2
3,29,1912.08.11,12:19:04 AM,40.60,27.20,south_east,30,4.9,4.3
4,38,1912.08.11,12:20:00 AM,40.60,27.10,south_west,15,4.5,2
...,...,...,...,...,...,...,...,...,...
10057,80461,2017.07.26,12:50:05 AM,40.76,32.87,south_east,5,3.7,2.1
10058,80465,2017.07.29,12:24:11 AM,37.12,28.59,north,5,3.9,0.5
10059,80474,2017.07.29,12:33:23 AM,37.11,27.73,north_east,6.5,3.5,1.2
10060,80486,2017.07.30,12:21:33 AM,37.39,31.37,south_west,5,3.5,6
